# Calibrating the CATS model against the US economy using black-it

<table style="font-size: 110%">
    <tr style="background-color: white;">
    <td width="50%" style="vertical-align: top; text-align: left">
        We will use black-it to calibrate CATS (Complex Adaptive Trivial System) a well-known macroeconomic model that our team reimplemented in Julia.<br><br>
        The high level structure of the model is illustrated on the right: <span style="background-color: #D3E7C6; font-weight: bold;">Agent classes</span> are represented in green ovals, interaction types are specified in rectangles: <span style="background-color: #FFE699; font-weight: bold;">markets</span> are yellow, <span style="background-color: #F8CBAD; font-weight: bold;">bank deposits</span> are peach.<br><br>
        The directions of the arrows indicate the flow of the specific good e.g., consumption goods are acquired by households from C-firms, while labour is acquired by firms from households.<br><br>
        The model has <strong>28 input parameters</strong> and <strong>16 output parameters</strong>.</td>
    <td style="vertical-align: top; text-align: left"><div style="text-align: center">CATS high level architecture:</div><img src="data/agenc-architecture.png" alt="AgenC architechture" style="width: 500px;"/></td>
    </tr>
</table>

In [ ]:
import pathlib
import shutil

path_to_model_output = pathlib.Path("").resolve() / "CATS_model_output"
print(f"Removing {path_to_model_output}")
shutil.rmtree(path_to_model_output, ignore_errors=True)

In [ ]:
import abm_models
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import cats_util

from black_it.calibrator import Calibrator
from black_it.loss_functions.msm import MethodOfMomentsLoss
from black_it.samplers.best_batch import BestBatchSampler
from black_it.samplers.random_forest import RandomForestSampler
from black_it.samplers.halton import HaltonSampler
from black_it.utils.time_series import get_mom_ts, get_mom_ts_1d
from black_it.utils.time_series import log_and_hp_filter, diff_log_demean_filter

In [ ]:
cats_model = abm_models.make(
    "CatsModel", 
    path_to_cats_directory="/home/black-it-experiments/disk/black-it-demo/AgenC",
    W=500,
    F=50,
    N=10, 
    warm_up_samples=200,
    output_variables=[
      "Y_real",
      "gdp_deflator",
      "Investment",
      "consumption",
      "Un",
    ]
)

In [ ]:
# import the target data, the US economy from 1947 to 2019
real_data = np.genfromtxt('FRED_data.txt')

In [ ]:
cats_util.plot_cats_output(real_data)

In [ ]:
# try to run the ABM with some parameters
output = cats_model.run(
    params={
        "Iprob": 0.25,
        "chi": 0.05,
        "delta": 0.5,
        "inventory_depreciation": 0.3,
        "mu": 1.2,
        "p_adj": 0.1,
        "phi": 0.02,
        "q_adj": 0.9,
        "tax_rate": 0.05,
        "theta": 0.05,
        "xi": 0.96,
    }, 
    nb_samples=281,
    seed=0,
)

In [ ]:
# plot one of the obtained series
plt.plot(output[:, 0])
plt.xlabel('quarters')
plt.ylabel('real gdp')

In [ ]:
# define a simple model wrapper
def model_wrapper(theta, N, seed):
    return cats_model.run(
        params=cats_util.found_params | {
            "Iprob": theta[0],
            "inventory_depreciation": theta[1],
            "mu": theta[2],
        },
        nb_samples=N, 
        seed=seed,
    )

In [ ]:
# define a method of moments loss function with some coordinate filters
coordinate_filters = [
    log_and_hp_filter, 
    diff_log_demean_filter, 
    log_and_hp_filter, 
    log_and_hp_filter, 
    None,
]

loss = MethodOfMomentsLoss(
    coordinate_filters=coordinate_filters,
    covariance_mat="identity",
    standardise_moments=True,
)

In [ ]:
# define the simulation bounds and precisions
bounds = np.array([
        [0.0, 0.5],
        [0.0, 1.0],
        [1.0, 1.5]]).T

precisions = [0.001] * 3

In [ ]:
# define a list of search methods, one just for seeding
seed_sampler = [HaltonSampler(batch_size=8)]
adaptive_sampler = [RandomForestSampler(batch_size=8), BestBatchSampler(batch_size=8)]

In [ ]:
# folder where data is saved
saving_folder = "CATS_model_output"

# length of the simulations
sim_length = 800
ensemble_size = 4

# initialize the Calibrator
cal = Calibrator(
    samplers=seed_sampler,
    real_data=real_data,
    sim_length=sim_length,
    model=model_wrapper,
    parameters_bounds=bounds,
    parameters_precision=precisions,
    ensemble_size=ensemble_size, 
    loss_function=loss,
    saving_folder=saving_folder,
    random_state=0,
    n_jobs=32,
)

In [ ]:
# sample some initial seed points uniformly using a Halton sampler
params, losses = cal.calibrate(1)

In [ ]:
# change the sampler to a more adaptive one
cal.set_samplers(adaptive_sampler)

In [ ]:
# calibrate for a number of epochs
cal.calibrate(5)

In [ ]:
# find the best loss index and print the best loss
best_idx = np.argmin(cal.losses_samp)
print(f"Best loss index: {best_idx} (out of {len(cal.losses_samp)} simulations)")
print(f"Best loss value: {cal.losses_samp[best_idx]:.2f}")

In [ ]:
# retrieve the best loss simulations
best_sim = cal.series_samp[best_idx]

In [ ]:
# compare real and simulated moments
fig, axes = plt.subplots(2, 5, figsize=(8, 3.5))

xlabels_kde = [[-0.1, 0.0, 0.1 ], [-0.05, 0, 0.05], [-0.3, 0, 0.3],[-0.1, 0, 0.1], [0., 0.1, 0.2], ]
clips = [(-0.1, 0.1), (-0.1, 0.1), (-0.4, 0.4), (-0.1, 0.1), (0.0, 0.25)]
cmap = plt.get_cmap("tab10")

for i, filter_ in enumerate(coordinate_filters):
    ### DATA ##:
    if filter_ is None:
        filter_ = lambda x: x
    filtered_data = np.array([filter_(best_sim[j, :, i]) for j in range(ensemble_size)])

    real_moments = get_mom_ts_1d(real_data[:, i])
    sim_moments = np.array([get_mom_ts_1d(filtered_data[j, :]) for j in range(ensemble_size)]) 
    m = np.mean(sim_moments, axis = 0)
    s = np.std(sim_moments, axis = 0)

    ### KDE ###
    ax = axes[0, i]
    ax.set_title(cats_util.target_labels[i])
    sns.kdeplot(ax=ax, data=real_data[:, i], label = 'target', 
                bw_adjust = 1., clip = clips[i], lw = 2)

    sns.kdeplot(ax=ax, data = filtered_data.flatten(),
                color = cmap(1), label = 'lowest loss', 
                bw_adjust = 1., clip = clips[i], lw = 2)

    ax.set_ylabel("")
    ax.set_yticklabels("")
    ax.set_xticks(xlabels_kde[i])

    ### MOMENTS ###
    index = np.arange(len(real_moments))
    ax = axes[1, i]
    ax.set_ylim(-1, 2.2)
    ax.plot(index, real_moments, '-o', ms = 3)

    ax.plot(index, m, color = cmap(1), ls = '-',marker = 'o', ms = 3)
    ax.fill_between(index, m - s, m + s, alpha = 0.25, color = cmap(1))
    ax.set_xticks([0, 4, 9, 13])
    ax.set_xlabel("")

axes[0, 0].set_ylabel("density", labelpad= 10)
axes[1, 0].set_ylabel("moments value", labelpad= -3)

for i in range(1, 5):
    axes[1, i].set_yticklabels("")

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.3,
                    hspace=0.3)

axes[0, 4].plot([0.1, 0.15],[21, 21], lw = 2, color = cmap(0))
axes[0, 4].text(0.1,18,"real")

axes[0, 4].plot([0.1, 0.15],[13, 13], lw = 2, color = cmap(1))
axes[0, 4].text(0.1,10,"simulated")

plt.show()

- 0-3: mean, variance, skewness and kurtosis
- 4-8: autocorrelations of increasing time lags
- 9-12: mean, variance, skewness and kurtosis of the differentiated time series
- 13-17: autocorrelations of the differentiated time series.